# Computer Vision: Gap Calculation using YOLO net and OpenCV

In [2]:
import cv2
import numpy as np

In [3]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import Counter
import time as pytime

# Utilizar GPU si está disponible
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Cargar el modelo Anti-Alpine optimizado
model = YOLO("../f1-strategy/weights/model_anti_alpine.pt")
model.to(DEVICE)

# Configuración de tamaño y escala
FRAME_WIDTH = 1280  # Mayor resolución para mejor detalle
CAR_LENGTH_METERS = 5.63  # Longitud real del coche en metros

# PARA GAPS: usar un umbral global más bajo para maximizar detecciones
GAP_DETECTION_THRESHOLD = 0.25  # Umbral bajo para detectar todos los coches posibles

# Colores específicos para cada equipo de F1 (formato BGR para OpenCV)
class_colors = {
    'Ferrari': (0, 0, 255),         # Rojo (BGR)
    'Mercedes': (200, 200, 200),    # Plateado (BGR)
    'Red Bull': (139, 0, 0),        # Azul oscuro (BGR)
    'McLaren': (0, 165, 255),       # Naranja (BGR)
    'Aston Martin': (0, 128, 0),    # Verde (BGR)
    'Alpine': (128, 0, 0),          # Azul (BGR)
    'Williams': (205, 0, 0),        # Azul claro (BGR)
    'Haas': (255, 255, 255),        # Blanco (BGR)
    'Kick Sauber': (255, 255, 0),   # Cian (BGR)
    'Racing Bulls': (0, 0, 255),    # Rojo (BGR)
    'background': (128, 128, 128),  # Gris (BGR)
    'unknown': (0, 255, 255)        # Amarillo para coches sin clasificación segura
}

# Umbrales para mostrar la clasificación (no para filtrar detecciones)
class_thresholds = {
    'Williams': 0.90,     # Muy alto para Williams
    'Alpine': 0.90,       # Extremadamente alto para Alpine
    'McLaren': 0.30,      # Bajo para McLaren
    'Red Bull': 0.85,     # Alto para Red Bull
    'Ferrari': 0.40,      # Normal
    'Mercedes': 0.50,     # Medio-alto
    'Haas': 0.40,         # Normal
    'Kick Sauber': 0.40,  # Normal
    'Racing Bulls': 0.40, # Normal
    'Aston Martin': 0.40, # Normal
    'background': 0.50    # Alto para fondo
}

# Historial de detecciones para estabilización
last_detections = {}
track_history = {}
id_counter = 0
class_history = {}

def calculate_gap(box1, box2, class1, class2):
    """Calcula la distancia entre centros usando el ancho de los coches para la escala"""
    # Centros de las cajas
    cx1, cy1 = (box1[0] + box1[2])/2, (box1[1] + box1[3])/2
    cx2, cy2 = (box2[0] + box2[2])/2, (box2[1] + box2[3])/2
    
    # Distancia en píxeles
    pixel_distance = np.hypot(cx2 - cx1, cy2 - cy1)
    
    # Escala basada en el ancho promedio de los coches detectados
    avg_width = ((box1[2] - box1[0]) + (box2[2] - box2[0])) / 2
    scale = CAR_LENGTH_METERS / avg_width if avg_width != 0 else 0
    
    # Calcular tiempo de gap a 300km/h (83.33 m/s)
    speed_mps = 83.33  # Metros por segundo a 300km/h
    gap_time = (pixel_distance * scale) / speed_mps
    
    return {
        'distance': pixel_distance * scale,  # Distancia en metros
        'time': gap_time,                   # Tiempo en segundos a 300km/h
        'car1': class1,                     # Equipo del primer coche
        'car2': class2                      # Equipo del segundo coche
    }

def process_video_with_yolo(video_path, output_path=None):
    global last_detections, track_history, id_counter, class_history, GAP_DETECTION_THRESHOLD
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return
    
    # Obtener dimensiones originales del video
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    current_frame = 0
    
    # Calcular nuevo alto manteniendo la relación de aspecto
    target_height = int(FRAME_WIDTH * original_height / original_width)
    
    # Configurar salida si se solicita
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (FRAME_WIDTH, target_height))
    
    # Variables para calcular FPS real
    frame_count = 0
    start_time = pytime.time()
    current_fps = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: 
            break
        
        frame_count += 1
        current_frame += 1
        
        # Redimensionar manteniendo la relación de aspecto
        frame_resized = cv2.resize(frame, (FRAME_WIDTH, target_height))
        original_frame = frame_resized.copy()
        
        # Ejecutar la detección con YOLOv8 con umbral bajo para maximizar detecciones
        results = model.predict(
            source=frame_resized, 
            conf=GAP_DETECTION_THRESHOLD,  # Umbral bajo para detectar todos los coches posibles
            iou=0.45,   # IoU estándar
            max_det=20, # Máximo de detecciones
            verbose=False
        )[0]
        
        # Detecciones actuales
        current_detections = {}
        
        # Procesar resultados de la detección
        if results.boxes and len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()
            class_ids = results.boxes.cls.cpu().numpy().astype(int)
            
            # Crear lista de detecciones con toda la información
            detections = []
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                conf = float(confs[i])
                class_id = int(class_ids[i])
                cls_name = model.names[class_id]
                
                # Determinar si mostrar la clasificación del equipo basado en umbral
                # Nota: seguimos detectando el coche pero podríamos no mostrar su equipo
                classified = conf >= class_thresholds.get(cls_name, 0.40)
                
                # CLAVE: Para gaps, detectamos todos los coches incluso si no estamos seguros del equipo
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                area = (x2 - x1) * (y2 - y1)
                
                # Asignar ID único o recuperar ID existente
                object_id = None
                for old_id, old_info in last_detections.items():
                    old_cx, old_cy = old_info['center']
                    old_cls = old_info['class']
                    
                    # Distancia entre centros
                    dist = np.sqrt((center_x - old_cx)**2 + (center_y - old_cy)**2)
                    
                    # Si está cerca, podría ser el mismo objeto
                    if dist < 100:
                        object_id = old_id
                        
                        # Si la clase anterior era válida y la nueva es incierta, mantener la anterior
                        if old_info['classified'] and not classified:
                            cls_name = old_cls
                            classified = True
                        
                        # Estabilizar la clasificación con el historial para clases problemáticas
                        if classified and old_cls != cls_name and (cls_name == 'Williams' or cls_name == 'Alpine'):
                            if old_info['classified']:
                                cls_name = old_cls
                        break
                
                # Si no se encontró correspondencia, asignar nuevo ID
                if object_id is None:
                    object_id = id_counter
                    id_counter += 1
                    track_history[object_id] = []
                    class_history[object_id] = []
                
                # Actualizar historial
                if object_id in class_history:
                    # Solo añadir al historial si estamos seguros de la clase
                    if classified:
                        class_history[object_id].append(cls_name)
                        # Limitar historial a 5 clases
                        if len(class_history[object_id]) > 5:
                            class_history[object_id].pop(0)
                    
                    # Usar la clase más común del historial para estabilidad
                    if len(class_history[object_id]) >= 3:
                        counts = Counter(class_history[object_id])
                        if counts:  # Asegurarse que no está vacío
                            most_common = counts.most_common(1)[0][0]
                            cls_name = most_common
                            classified = True
                
                # Guardar detección actual
                current_detections[object_id] = {
                    'box': (x1, y1, x2, y2),
                    'conf': conf,
                    'class': cls_name,
                    'classified': classified,
                    'center': (center_x, center_y),
                    'area': area,
                    'y_bottom': y2  # Para ordenar por posición vertical
                }
                
                # Añadir a lista de detecciones para cálculo de gaps
                detections.append({
                    'id': object_id,
                    'box': (x1, y1, x2, y2),
                    'class': cls_name,
                    'classified': classified,
                    'conf': conf,
                    'y_bottom': y2
                })
            
            # Ordenar por posición vertical (coches más abajo primero - más cercanos)
            detections = sorted(detections, key=lambda x: x['y_bottom'], reverse=True)
            
            # Dibujar cajas y gaps
            for i, det in enumerate(detections):
                x1, y1, x2, y2 = det['box']
                cls_name = det['class']
                conf = det['conf']
                classified = det['classified']
                
                # Obtener color específico para el equipo
                if classified:
                    color = class_colors.get(cls_name, (0, 255, 0))
                else:
                    color = class_colors['unknown']  # Amarillo para coches sin clasificación segura
                
                # Dibujar caja con color del equipo
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), color, 2)
                
                # Etiqueta con clase y confianza
                if classified:
                    label = f"{cls_name}: {conf:.2f}"
                else:
                    label = f"F1 Car: {conf:.2f}"
                
                t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                cv2.rectangle(frame_resized, (x1, y1-t_size[1]-3), (x1+t_size[0], y1), color, -1)
                cv2.putText(frame_resized, label, (x1, y1-3), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Solo si hay siguiente coche
                if i < len(detections)-1:
                    next_det = detections[i+1]
                    gap_info = calculate_gap(
                        det['box'], next_det['box'], 
                        det['class'] if det['classified'] else "F1 Car", 
                        next_det['class'] if next_det['classified'] else "F1 Car"
                    )
                    
                    # Puntos de conexión
                    cx1, cy1 = int((x1+x2)/2), int(y1)  # Usar parte superior del coche
                    nx1, ny1, nx2, ny2 = next_det['box']
                    cx2, cy2 = int((nx1+nx2)/2), int(ny2)  # Usar parte inferior del siguiente
                    
                    # Línea diagonal entre coches
                    cv2.line(frame_resized, (cx1, cy1), (cx2, cy2), (0, 255, 0), 2)
                    
                    # Texto en el punto medio con más información
                    mid_x, mid_y = (cx1+cx2)//2, (cy1+cy2)//2
                    
                    # Distancia y tiempo de gap
                    dist_text = f"{gap_info['distance']:.1f}m"
                    time_text = f"{gap_info['time']:.2f}s"
                    
                    # Fondo para el texto
                    dist_size = cv2.getTextSize(dist_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    time_size = cv2.getTextSize(time_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    
                    # Dibujar fondo semitransparente
                    overlay = frame_resized.copy()
                    cv2.rectangle(overlay, 
                                 (mid_x - 5, mid_y - 50), 
                                 (mid_x + max(dist_size[0], time_size[0]) + 10, mid_y + 10),
                                 (0, 0, 0), -1)
                    cv2.addWeighted(overlay, 0.6, frame_resized, 0.4, 0, frame_resized)
                    
                    # Dibujar textos
                    cv2.putText(frame_resized, dist_text, (mid_x, mid_y - 25),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    cv2.putText(frame_resized, time_text, (mid_x, mid_y),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Actualizar last_detections para la próxima iteración
        last_detections = current_detections
        
        # Calcular FPS
        if frame_count % 10 == 0:
            current_time = pytime.time()
            current_fps = 10.0 / (current_time - start_time)
            start_time = current_time
        
        # Mostrar FPS e información del modelo
        cv2.putText(frame_resized, f"FPS: {current_fps:.1f}", (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.putText(frame_resized, "F1 Gap Detection", (FRAME_WIDTH - 300, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Mostrar modo de detección y progreso
        detection_mode = f"Detection Threshold: {GAP_DETECTION_THRESHOLD:.2f}"
        cv2.putText(frame_resized, detection_mode, (20, target_height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Mostrar progreso del video
        progress_text = f"Frame: {current_frame}/{total_frames} ({current_frame/total_frames*100:.1f}%)"
        cv2.putText(frame_resized, progress_text, (FRAME_WIDTH - 400, target_height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Guardar frame procesado si se solicitó
        if output_path:
            out.write(frame_resized)
        
        # Mostrar frame
        cv2.imshow("F1 Gap Detection", frame_resized)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('+'):  # Aumentar umbral
            GAP_DETECTION_THRESHOLD = min(GAP_DETECTION_THRESHOLD + 0.05, 0.95)
            print(f"Detection threshold increased to {GAP_DETECTION_THRESHOLD:.2f}")
        elif key == ord('-'):  # Disminuir umbral
            GAP_DETECTION_THRESHOLD = max(GAP_DETECTION_THRESHOLD - 0.05, 0.05)
            print(f"Detection threshold decreased to {GAP_DETECTION_THRESHOLD:.2f}")
        elif key == ord('d'):  # Avanzar 10 segundos
            skip_frames = int(fps * 10)  # 10 segundos * fps = número de frames a saltar
            new_frame_pos = min(current_frame + skip_frames, total_frames - 1)
            cap.set(cv2.CAP_PROP_POS_FRAMES, new_frame_pos)
            current_frame = new_frame_pos - 1  # Se incrementará en el próximo ciclo
            # Reiniciar tracking temporalmente
            last_detections = {}
            print(f"Skipped forward 10 seconds to frame {new_frame_pos}")
    
    # Liberar recursos
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Ejecutar con tu video
def main():
    #video_path = "../f1-strategy/data/videos/best_overtakes_2023.mp4.f399.mp4"
    #video_path = "../f1-strategy/data/videos/spain_2023_race.mp4.f399.mp4"
    
    video_path = "../f1-strategy/data/videos/belgium_gp.f399.mp4"
    output_path = "../f1-strategy/data/videos/gap_detection_output.mp4"
    process_video_with_yolo(video_path, output_path)

if __name__ == "__main__":
    main()

Skipped forward 10 seconds to frame 570
Skipped forward 10 seconds to frame 1124
Skipped forward 10 seconds to frame 1655


Controls

'q': out
'+': more detection threshold
'-': less detection threshold
'd': 10 seconds ahead 